# 0.4_ss_logistic
ロジスティック回帰頑張ろ

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:

import logging
log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_fmt)
# logging.basicConfig(level=logging.DEBUG, format=log_fmt)
logger = logging.getLogger()

## Logistic one loop
- 1 loop だけ回す

In [18]:
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
class LogisticRegression:

In [23]:
impute = True
as_numpy = True

In [25]:
def load_breast_cancer(impute=True, as_numpy=True):
    url = ('https://raw.githubusercontent.com/selva86/'
           'datasets/master/BreastCancer.csv')
    df = pd.read_csv(url)
    y = df.Class
    X = df.drop(['Class'], 1)

    imp = SimpleImputer()
    if impute:
        X = imp.fit_transform(X)

    if as_numpy:
        X = np.array(X)
        y = np.array(y)
    return train_test_split(X, y)

X_train, X_test, y_train, y_test = load_breast_cancer() 

In [8]:
tol = 0.001
max_iter = 3
random_seed = 0
random_state = np.random.RandomState(random_seed)

In [29]:
X = X_train

In [30]:
w_ = random_state.randn(X.shape[1]+1)
w_.shape

(11,)

In [31]:
w_

array([ 0.8644362 , -0.74216502,  2.26975462, -1.45436567,  0.04575852,
       -0.18718385,  1.53277921,  1.46935877,  0.15494743,  0.37816252,
       -0.88778575])

In [33]:
Xtil = np.c_[np.ones(X.shape[0]), X]
Xtil

array([[1.000000e+00, 1.311108e+06, 1.000000e+00, ..., 1.000000e+00,
        1.000000e+00, 1.000000e+00],
       [1.000000e+00, 4.769030e+05, 1.000000e+01, ..., 3.000000e+00,
        3.000000e+00, 8.000000e+00],
       [1.000000e+00, 1.368267e+06, 5.000000e+00, ..., 1.000000e+00,
        1.000000e+00, 1.000000e+00],
       ...,
       [1.000000e+00, 1.352663e+06, 5.000000e+00, ..., 8.000000e+00,
        1.000000e+01, 1.000000e+00],
       [1.000000e+00, 1.155967e+06, 5.000000e+00, ..., 2.000000e+00,
        1.000000e+00, 1.000000e+00],
       [1.000000e+00, 4.881730e+05, 1.000000e+00, ..., 5.000000e+00,
        6.000000e+00, 1.000000e+00]])

In [34]:
X

array([[1.311108e+06, 1.000000e+00, 1.000000e+00, ..., 1.000000e+00,
        1.000000e+00, 1.000000e+00],
       [4.769030e+05, 1.000000e+01, 5.000000e+00, ..., 3.000000e+00,
        3.000000e+00, 8.000000e+00],
       [1.368267e+06, 5.000000e+00, 1.000000e+00, ..., 1.000000e+00,
        1.000000e+00, 1.000000e+00],
       ...,
       [1.352663e+06, 5.000000e+00, 4.000000e+00, ..., 8.000000e+00,
        1.000000e+01, 1.000000e+00],
       [1.155967e+06, 5.000000e+00, 1.000000e+00, ..., 2.000000e+00,
        1.000000e+00, 1.000000e+00],
       [4.881730e+05, 1.000000e+00, 4.000000e+00, ..., 5.000000e+00,
        6.000000e+00, 1.000000e+00]])

In [36]:
diff = np.inf
w_prev = w_
iter_ = 0

In [37]:
assert(diff > tol and iter_ < max_iter)

In [38]:
yhat = sigmoid(np.dot(Xtil, w_))

/Users/shotashimizu/.pyenv/versions/3.7.0/envs/essence/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [54]:
type(Xtil)

numpy.ndarray

In [53]:
type(w_)

numpy.ndarray

In [51]:
type(yhat)

numpy.ndarray

In [41]:
THRESMIN = 1e-10

In [49]:
r = np.clip(yhat * (1-yhat), THRESMIN, np.inf)
logger.info(f"r: {r.shape}")

2018-12-03 23:15:41,721 - root - INFO - r: (524,)


In [50]:
type(r)

numpy.ndarray

In [48]:
XR = Xtil.T * r
logger.info(f"XR: {XR.shape}, Xtil.T: {Xtil.T.shape}")

2018-12-03 23:15:21,637 - root - INFO - XR: (11, 524), Xtil.T: (11, 524)


# Loop

In [58]:
class LogisticRegression:
    def __init__(self, tol=0.001, max_iter=3, random_seed=0):
        self.tol = tol
        self.max_iter = max_iter
        self.random_state = np.random.RandomState(random_seed)
        self.w_ = None
    
    def fit(self, X, y):
        assert(type(X) == np.ndarray)
        assert(type(y) == np.ndarray)
        
        self.w_ = self.random_state.randn(X.shape[1] + 1)
        Xtil = np.c_[np.ones(X.shape[0]), X]
        diff = np.inf
        w_prev = self.w_
        iter_ = 0
        THRESMIN = 1e-10 
        while diff > self.tol and iter_ < self.max_iter:
            yhat = sigmoid(np.dot(Xtil, self.w_))
            r = np.clip(yhat*(1-yhat), THRESMIN, np.inf)
            XR = Xtil.T * r
            XRX = np.dot(XR, Xtil)
            w_prev = self.w_
            b = np.dot(XR, np.dot(Xtil, self.w_) - 1/r * (yhat - y))
            self.w_ = linalg.solve(XRX, b)
            diff = abs(w_prev - self.w_).mean()
            iter_ += 1
    
    def predict(self, X):
        assert(type(X) == np.ndarray)
        
        Xtil = np.c_[np.ones(X.shape[0]), X]
        yhat = sigmoid(np.dot(Xtil, self.w_))
        return np.where(yhat > .5, 1, 0)

In [66]:
X_train, X_test, y_train, y_test = load_breast_cancer()
n_test = y_test.shape[0]

In [72]:
model = LogisticRegression(tol=0.001)

In [73]:
model.fit(X_train, y_train)

/Users/shotashimizu/.pyenv/versions/3.7.0/envs/essence/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [74]:
y_predict = model.predict(X_test)

/Users/shotashimizu/.pyenv/versions/3.7.0/envs/essence/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [75]:
n_hits = (y_test == y_predict).sum()

In [76]:
logger.info(f"Accuracy: {n_hits}/{n_test} = {n_hits/n_test}")

2018-12-04 00:08:04,117 - root - INFO - Accuracy: 163/175 = 0.9314285714285714
